## ANALIZA

##Spajanje tabela

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

In [2]:
url = 'https://www.worldometers.info/gdp/gdp-by-country/'
request = get(url)

In [3]:
medals = pd.read_csv ('medals_for_analysis.csv')

In [4]:
country_table = pd.read_csv ('country_for_analysis.csv')

In [5]:
medals_grouped = medals.groupby(['country']).size().reset_index(name = 'count')

In [6]:
medals_grouped.head()

,country,count
0,Albania,2
1,Algeria,3
2,Argentina,3
3,Armenia,4
4,Australia,53


In [7]:
merged = medals_grouped.merge(country_table, how = 'inner', on = 'country')

In [8]:
merged.head()

,country,count,Unnamed: 0,gdp,population,gdp_per_capita
0,Albania,2,117,1.888210e+10,2827608.0,6678.0
1,Algeria,3,54,1.919130e+11,45477389.0,4220.0
2,Argentina,3,21,6.327700e+11,45407904.0,13935.0
3,Armenia,4,114,1.950278e+10,2880874.0,6770.0
4,Australia,53,11,1.675420e+12,26200984.0,63945.0


# Correlation

In [9]:
numeric = merged.select_dtypes(include = 'number')
numeric.corr()

,count,Unnamed: 0,gdp,population,gdp_per_capita
count,1.000000,-0.455384,0.858858,0.392787,0.341684
Unnamed: 0,-0.455384,1.000000,-0.351191,-0.312197,-0.454880
gdp,0.858858,-0.351191,1.000000,0.570042,0.229968
population,0.392787,-0.312197,0.570042,1.000000,-0.102252
gdp_per_capita,0.341684,-0.454880,0.229968,-0.102252,1.000000


In [12]:
merged['medals per gdp'] = round(merged ['count'] * 100000000000/merged['gdp'])
merged.sort_values(by = 'medals per gdp', ascending = False).reset_index()

,index,country,count,Unnamed: 0,gdp,population,gdp_per_capita,medals per gdp
0,23,Dominica,1,171,6.120481e+08,66826.0,9159.0,163.0
1,33,Grenada,2,165,1.256413e+09,116913.0,10747.0,159.0
2,65,Saint Lucia,2,160,2.065028e+09,178781.0,11551.0,97.0
3,47,Kyrgyzstan,6,140,1.093064e+10,6955788.0,1571.0,55.0
4,12,Cabo Verde,1,158,2.314817e+09,519741.0,4454.0,43.0
...,...,...,...,...,...,...,...,...
80,59,Peru,1,50,2.426320e+11,33475438.0,7248.0,0.0
81,67,Singapore,1,32,4.667890e+11,5649885.0,82619.0,0.0
82,63,Qatar,1,51,2.372960e+11,2892455.0,82040.0,0.0
83,57,Pakistan,1,40,3.765330e+11,243700667.0,1545.0,0.0


In [13]:
merged.describe()

,count,Unnamed: 0,gdp,population,gdp_per_capita,medals per gdp
count,85.000000,85.000000,8.500000e+01,8.500000e+01,85.000000,85.000000
mean,11.964706,59.541176,1.068165e+12,7.250383e+07,22539.847059,10.741176
std,20.272810,45.592803,3.395509e+12,2.199779e+08,25479.218301,27.256783
min,1.000000,0.000000,6.120481e+08,6.682600e+04,963.000000,0.000000
25%,2.000000,23.000000,6.350175e+10,5.456801e+06,4965.000000,1.000000
50%,5.000000,51.000000,2.372960e+11,1.211933e+07,10996.000000,2.000000
75%,10.000000,82.000000,5.792670e+11,5.173794e+07,29367.000000,8.000000
max,126.000000,177.000000,2.546270e+13,1.425423e+09,106155.000000,163.000000


In [14]:
gdp_bins = [0, 60000000000, 200000000000, 600000000000, float('inf')]
gdp_labels = ['Low GDP', 'Lower Middle GDP', 'Upper Middle GDP', 'High GDP']

In [15]:
merged['GDP_category'] = pd.cut(merged['gdp'], bins = gdp_bins, labels = gdp_labels)

In [17]:
merged.head()

,country,count,Unnamed: 0,gdp,population,gdp_per_capita,medals per gdp,GDP_category
0,Albania,2,117,1.888210e+10,2827608.0,6678.0,11.0,Low GDP
1,Algeria,3,54,1.919130e+11,45477389.0,4220.0,2.0,Lower Middle GDP
2,Argentina,3,21,6.327700e+11,45407904.0,13935.0,0.0,High GDP
3,Armenia,4,114,1.950278e+10,2880874.0,6770.0,21.0,Low GDP
4,Australia,53,11,1.675420e+12,26200984.0,63945.0,3.0,High GDP
...,...,...,...,...,...,...,...,...
80,Ukraine,12,57,1.605030e+11,41048766.0,3910.0,7.0,Lower Middle GDP
81,United Kingdom,65,5,3.070670e+12,68179315.0,45038.0,2.0,High GDP
82,United States,126,0,2.546270e+13,341534046.0,74554.0,0.0,High GDP
83,Uzbekistan,13,69,8.039185e+10,34938955.0,2301.0,16.0,Lower Middle GDP


In [18]:
gdp_medal_summary = merged.groupby('GDP_category')['count'].sum().reset_index()

C:\Users\sne sta\AppData\Local\Temp\ipykernel_148524\1881112692.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gdp_medal_summary = merged.groupby('GDP_category')['count'].sum().reset_index()


In [19]:
gdp_medal_summary

,GDP_category,count
0,Low GDP,54
1,Lower Middle GDP,119
2,Upper Middle GDP,153
3,High GDP,691
